## RetinaNet

Keras-RetinaNet 모델 훈련 및 예측 과정입니다. [keras-retinanet](https://github.com/fizyr/keras-retinanet) 패키지가 필요합니다.

 - Tensorflow를 다운로드 및 설치합니다. 2.3.0 이후 버전이 필요합니다.
 ```
 python -m pip install tensorflow
 ```
 - Git 저장소에서 최신 패키지를 다운로드 및 설치합니다.
 ```
 git clone https://github.com/fizyr/keras-retinanet.git
 cd keras-retinanet
 python -m pip install .
 ```

### Train

keras-retinanet 패키지는 커맨드라인 상에서 훈련을 할 수 있는 명령어를 스크립트 형태로 제공합니다. `retinanet-train` 명령어를 사용하시면 됩니다.

 - 기본 사용법: `retinanet-train (옵션) (데이터셋 종류) (데이터셋 경로) (데이터셋 옵션)`
   - 데이터셋 종류는 `coco`, `pascal`, `csv`, `oid`, `kitti` 5종류가 있습니다.
   - coco 데이터셋은 [pycocotools](https://pypi.org/project/pycocotools/) 의존이 추가로 필요합니다.
   - 커스텀 데이터셋에서 훈련을 하고자 하는 경우 csv 데이터셋 사용을 권장합니다.
 - csv 데이터셋의 경우: `retinanet-train csv (어노테이션 파일 경로) (클래스 파일 경로)`
   - 어노테이션 CSV 파일은 `이미지/경로.jpg,x1,y1,x2,y2,클래스이름` 양식으로 각 경계 상자에 대한 정보를 기술합니다.
   - 이미지 경로를 제외한 정보를 생략할 경우 (`이미지/경로.jpg,,,,,`) 음성 샘플로 간주되어 훈련에 사용됩니다.
   - 클래스 CSV 파일은 `클래스이름,인덱스` 양식으로 각 클래스 이름과 인덱스의 대응 관계에 대한 정보를 기술합니다.
   - CSV 데이터셋 추가 옵션:
     - `--val-annotations (경로)` 검증용 데이터셋에 대한 어노테이션 파일 경로를 지정합니다.
 - 공통 추가 옵션
   - `--no-evaluation` 이 옵션을 사용하면 검증을 수행하지 않습니다. 사용하지 않으면 검증용 데이터셋이 존재할 경우 검증을 수행합니다.
   - `--weights (경로)` 기존 저장된 모델 스냅샷 정보를 불러와 가중치를 해당 모델의 값으로 초기화합니다.
   - `--no-weights` 가중치를 특정 값 대신 무작위로 초기화합니다. 가중치 관련 옵션이 지정되지 않은 경우 ImageNet 사전 훈련(pre-trained) 가중치를 사용합니다.
   - `--snapshot-path (경로)` 모델 정보가 저장될 폴더를 지정합니다. 기본값은 `./snapshot/`입니다.
   - `--tensorboard-dir (경로)` 텐서보드용 로그가 저장될 폴더를 지정합니다. 지정하지 않으면 텐서보드 로그가 저장되지 않습니다.
   - `--backbone (이름)` 레티나넷에 사용될 백본 CNN 망을 지정합니다. 기본값은 논문에서 사용된 백본 망인 `resnet50`입니다.
   - `--epochs (숫자)` 총 몇 세대(epoch)동안 훈련할 것인지를 정의합니다. 기본값은 `50`입니다.
   - `--steps (숫자)` 한 세대(epoch)동안 몇 배치가 훈련될 것인지를 정의합니다. 기본값은 `10000`입니다. 훈련 데이터셋 길이를 배치 사이즈로 나눈 값을 지정하는 것을 권장합니다.
   - `--batch-size (숫자)` 한번에 훈련할 이미지 갯수인 배치 사이즈를 지정합니다. 기본값은 `1`입니다.
   - `--gpu (숫자)` 몇번째 GPU를 훈련에 사용할지 정합니다.
 - 명령어 예시
   - `retinanet-train --steps 500 --gpu 0 csv dataset/train.csv dataset/cat.csv --val-annotations dataset/val.csv`

### Predict

In [ ]:
import cv2

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.visualization import draw_box, draw_caption

#### 데이터셋 설정

아래는 데이터셋 및 객체 검출 설정에 필요한 환경 변수입니다. 사용하는 데이터셋 및 모델에 맞춰서 변경하시기 바랍니다.

 - `classes` : 데이터내에 구성하는 모든 클래스의 정보를 담은 dict입니다. 키는 클래스의 일련변호, 값은 각 클래스의 이름으로 지정해야 합니다.
 - `weight_file_path` : 입력 weight 파일의 경로 및 이름을 지정합니다.
 - `predict_source` : 객체 검출을 할 대상 이미지 파일이 있는 디렉토리를 지정합니다.
 - `predict_result` : 객체 검출 결과가 출력될 디렉토리를 지정합니다. 자동으로 해당 디렉토리를 생성하지 않으므로 미리 생성해두셔야 합니다.
 - `bbox_threshold` : 검출된 경계 상자를 양성(positive)로 판정할 임계값입니다.
 - `resize` : 이미지가 지나치게 크면 리사이즈하여 검출할지를 지정하는 부울 값입니다.

In [ ]:
classes = {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle',
           5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow',
           10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person',
           15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}
model_path = './retinanet_predict.h5'
predict_source = './samples'
predict_result = './results'
bbox_threshold = 0.5

#### 검출 시작

모델을 로드하고 소스 이미지의 객체 검출을 실행하며 그 결과를 출력합니다.

In [ ]:
model = models.load_model(model_path)
model = models.convert_model(model)

for dir_path, _, filenames in os.walk(predict_source):
    for filename in filenames:
        if not filename.endswith(('.jpg', '.png')):
            continue

        # 이미지 로드
        start = time.time()
        file_path = os.path.join(dir_path, filename)
        print(file_path)
        image = read_image_bgr(file_path)
        draw = image.copy()

        # 정규화 및 리사이즈
        image = preprocess_image(image)
        image, scale = resize_image(image)

        # 검출
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
        boxes /= scale

        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            if score < bbox_threshold:
                break

            b = box.astype(int)

            draw_box(draw, b, label_color(label))
            draw_caption(draw, b, f'{classes[label]} {score:.3f}')
            print(f'Label: {classes[label]}, Score: {score:.3f}, LTRB of the boundary box: {b[0]}, {b[1]}, {b[2]}, {b[3]}')

        cv2.imwrite(os.path.join(predict_result, filename), image)

        print(f'Processing time: {time.time() - start}')

#### 모델 변환

keras-retinanet은 훈련 결과 가중치 스냅샷을 용량이 적은 예측 전용 모델로 변환할 수 있는 기능을 제공합니다. `retinanet-convert-model` 명령어를 사용하면 됩니다.
 - 기본 사용법: `retinanet-convert-model (훈련 모델 파일 경로) (출력될 예측 모델 파일 경로)`
 - 변환한 예측 전용 모델을 사용할 경우 위의 검출 스크립트에서 `model = models.convert_model(model)`을 주석 처리하거나 삭제하고 사용합니다.